In [21]:
import MySQLdb, re, time

def id2ensembl(id_list, 
                   genus_name, 
                   species_name, 
                   ensemblver=None):
    
    ''' Converts a list of any IDs into ensemblIDs
    
    Requires a list of IDs, a genus and species name, and
    optionally the enembl version to use. Genus_name and
    species_name must be the same that ensembl uses.
    
    For each matching record the script returns a list of tuples containing:
    (original ID, database ID matches, ensembl gene ID, biotype, description,
    chromosome, start coord, stop coord, strand).
    
    Example:
    
    ensembl_genes = id2ensembl(["SCFD2", "PPAP2C", "LASS6"],
                                   genus_name="homo",
                                   species_name="sapiens")
                                   
    original published here: https://gist.github.com/3452151
    update published here: https://gist.github.com/3487753
    '''
    
    # ensemlb details
    ensembl_host = "ensembldb.ensembl.org"
    ensembl_port = 5306
    
    # open connection
    ensemblDB = MySQLdb.connect(host=ensembl_host, 
                                user="anonymous", 
                                port=ensembl_port)
    cur = ensemblDB.cursor()
    
    # build ensembl species string
    ensembl_species = "%s_%s" % (genus_name.lower(), species_name.lower())
    
    # get tables in the schema matching the query species
    schema_sql = 'show schemas like "%s_core%s_%s"' % (ensembl_species,
                                                       "\\",
                                                       "%")
    cur.execute(schema_sql)
    records = cur.fetchall()
    print(records)
    
    if ensemblver is None:
        # trim out the latest one and split the version number 
        # and table version number
        latest_ver = records[-1][0].split("_")[-2:]
    else:
        for record in records:
            if re.match("%s_core_%i.+" % (ensembl_species, 
                                          ensemblver),
                        record[0]):
                latest_ver = record[0].split("_")[-2:]
    
    # define dbs to use
    core_db = "%s_core_%s_%s" % (ensembl_species, 
                                 latest_ver[0], 
                                 latest_ver[1])
    
    # id query string 
    # 'external_db_id=1300' = EntrezGene
    query_str = '''
        SELECT 
            x.display_label AS ID,
            GROUP_CONCAT(xdb.db_display_name) AS IDmatches,
            IFNULL(g.stable_id, pg.stable_id) AS eGene,
            IFNULL(g.biotype, pg.biotype) AS type,
            IFNULL(g.description, pg.description) AS gdesc,
            c.name AS chr,
            IFNULL(g.seq_region_start, pg.seq_region_start) AS start,
            IFNULL(g.seq_region_end, pg.seq_region_end) AS end,
            IFNULL(g.seq_region_strand, pg.seq_region_strand) AS strand    
        FROM %s.xref x 
        JOIN %s.external_db xdb ON x.external_db_id=xdb.external_db_id 
        LEFT JOIN %s.dependent_xref dx ON x.xref_id=dx.dependent_xref_id 
        LEFT JOIN %s.object_xref ox ON dx.object_xref_id=ox.object_xref_id
        LEFT JOIN %s.gene g ON ox.ensembl_id=g.gene_id 
        LEFT JOIN %s.gene pg ON pg.display_xref_id=x.xref_id
        JOIN %s.seq_region c ON g.seq_region_id=c.seq_region_id
        JOIN %s.coord_system cs ON c.coord_system_id=cs.coord_system_id
        WHERE cs.rank=1
        AND x.display_label IN ("%s")
        GROUP BY eGene;
        ''' % (core_db, core_db, core_db,
               core_db, core_db, core_db,
               core_db, core_db, '", "'.join(id_list)
               )
    
    #execute query
    ttime = time.gmtime()
    timestring =  "%02.0d:%02.0d:%02.0d" % (ttime[3],ttime[4],ttime[5])
    print("querying the ensembl database for all ensembl genes at %s..." % timestring)
    t1=time.time()
    cur.execute(query_str) # this is not so quick (~5min)
    records=cur.fetchall()
    print ("returned %i records in %.2fs" % (len(records), (time.time()-t1)))
    
    return(records)

In [22]:
ensembl_genes = id2ensembl(["SCFD2", "PPAP2C", "LASS6"],
                           genus_name="homo",species_name="sapiens")

(('homo_sapiens_core_100_38',), ('homo_sapiens_core_101_38',), ('homo_sapiens_core_102_38',), ('homo_sapiens_core_103_38',), ('homo_sapiens_core_104_38',), ('homo_sapiens_core_48_36j',), ('homo_sapiens_core_49_36k',), ('homo_sapiens_core_50_36l',), ('homo_sapiens_core_51_36m',), ('homo_sapiens_core_52_36n',), ('homo_sapiens_core_53_36o',), ('homo_sapiens_core_54_36p',), ('homo_sapiens_core_55_37',), ('homo_sapiens_core_56_37a',), ('homo_sapiens_core_57_37b',), ('homo_sapiens_core_58_37c',), ('homo_sapiens_core_59_37d',), ('homo_sapiens_core_60_37e',), ('homo_sapiens_core_61_37f',), ('homo_sapiens_core_62_37g',), ('homo_sapiens_core_63_37',), ('homo_sapiens_core_64_37',), ('homo_sapiens_core_65_37',), ('homo_sapiens_core_66_37',), ('homo_sapiens_core_67_37',), ('homo_sapiens_core_68_37',), ('homo_sapiens_core_69_37',), ('homo_sapiens_core_70_37',), ('homo_sapiens_core_71_37',), ('homo_sapiens_core_72_37',), ('homo_sapiens_core_73_37',), ('homo_sapiens_core_74_37',), ('homo_sapiens_core_

In [18]:
import pandas as pd
ensembl_genes = id2ensembl(
            pd.read_csv('../scClass_data/ref_gene.csv').values.flatten(),
            genus_name="homo",
            species_name="sapiens"
)

querying the ensembl database for all ensembl genes at 08:37:45...
returned 17568 records in 14.84s


In [35]:
ref_gene = pd.read_csv('../scClass_data/ref_gene.csv').values.flatten()

In [19]:
ensembl_genes

(('TSPAN6',
  'NCBI gene,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,WikiGene',
  'ENSG00000000003',
  'protein_coding',
  'tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]',
  'X',
  100627108,
  100639991,
  -1),
 ('DPM1',
  'NCBI gene,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,WikiGene',
  'ENSG00000000419',
  'protein_coding',
  'dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]',
  '20',
  50934867,
  50958555,
  -1),
 ('SCYL3',
  'NCBI gene,UniProtKB Gene Name,UniProtKB Gene Name,WikiGene',
  'ENSG00000000457',
  'protein_coding',
  'SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]',
  '1',
  169849631,
  169894267,
  -1),
 ('C1orf112',
  'NCBI gene,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,UniProtKB Gene Name,WikiGene',
  'ENSG00000000460',
  'protein_coding',
  'chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565

In [23]:
from pyensembl import EnsemblRelease

In [26]:
data = EnsemblRelease(78)
exon_ids  = data.exon_ids_of_gene_name('HLA-A')

In [67]:
ensg = []
i = 0 
for i,gene in enumerate(ref_gene):
    try:
        ensg.append(data.gene_ids_of_gene_name(gene))
    except:
        i = i+1
        ensg.append([''])
        

In [68]:
ensg

[[''],
 [''],
 ['ENSG00000225880'],
 ['ENSG00000230368'],
 ['ENSG00000188976'],
 ['ENSG00000187961'],
 ['ENSG00000187583'],
 [''],
 ['ENSG00000188290'],
 ['ENSG00000187608'],
 ['ENSG00000188157'],
 ['ENSG00000131591'],
 ['ENSG00000162571'],
 ['ENSG00000186891'],
 ['ENSG00000186827'],
 ['ENSG00000078808'],
 ['ENSG00000176022'],
 [''],
 [''],
 ['ENSG00000160087'],
 ['ENSG00000162572'],
 ['ENSG00000131584'],
 ['ENSG00000169972'],
 [''],
 ['ENSG00000224051'],
 ['ENSG00000169962'],
 ['ENSG00000107404'],
 ['ENSG00000162576'],
 ['ENSG00000175756'],
 ['ENSG00000221978'],
 ['ENSG00000242485'],
 [''],
 ['ENSG00000235098'],
 ['ENSG00000215915'],
 ['ENSG00000160072'],
 ['ENSG00000197785'],
 ['ENSG00000205090'],
 ['ENSG00000160075'],
 ['ENSG00000279244'],
 [''],
 [''],
 ['ENSG00000197530'],
 ['ENSG00000189409'],
 ['ENSG00000248333'],
 [''],
 ['ENSG00000189339'],
 ['ENSG00000008128'],
 [''],
 ['ENSG00000008130'],
 ['ENSG00000078369'],
 [''],
 ['ENSG00000067606'],
 [''],
 [''],
 [''],
 ['ENSG00000157

In [69]:
len(ensg)

20729

In [74]:
i=0
for e in ensg:
    if e==['']:
        i= i+1

In [75]:
i

4534

In [14]:
from sklearn.metrics import f1_score
y_true = [0,0,0,0,0,1,1,1,1,1]
y_pred = [0,0,0,1,1,0,1,1,1,1]
TP = 4
TN = 3
FP = 2
FN = 1
presion = TP/(TP+FP)
recall  = TP/(TP+FN)
F1 = 2*(presion*recall)/(presion+recall)
print('presion',presion)
print('recall',recall)
print('F1',F1)

presion 0.6666666666666666
recall 0.8
F1 0.7272727272727272


In [11]:
f1_score(y_true, y_pred, average=None)

array([0.66666667, 0.72727273])

In [15]:
f1_score(y_true, y_pred, average='macro')

0.6969696969696968